# Manipulation, SKOSification et Export Arborescent d'un Thésaurus CSV

Ce carnet réalise les étapes suivantes :
- Lecture d'un fichier CSV de thésaurus (structure ISO 25964)
- Construction d'un graphe SKOS (via rdflib)
- Visualisation de l'arbre et du graphe de concepts (networkx, matplotlib)
- Export d'un nouveau CSV avec l'arborescence explicite, une colonne par niveau

⚠️ Adaptez les chemins de fichiers selon vos besoins.


In [15]:
# Imports nécessaires
import csv
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import SKOS, RDF
import networkx as nx
import pandas as pd
import tkinter as tk
import matplotlib.pyplot as plt
%matplotlib qt


## 1. Paramètres et fonctions utilitaires

In [5]:
# Chemin du fichier CSV d'entrée et des résultats
DATA_DIR = "data"  # À adapter
CSV_FILE = f"{DATA_DIR}/20250710_BIB.csv"  # À adapter
SKOS_FILE = f"{DATA_DIR}/thesaurus.ttl"
ARBO_CSV = f"{DATA_DIR}/arborescence_niveaux.csv"

# Namespaces SKOS/EXEMPLE
EX = Namespace("http://example.org/mae/")
THESAURUS = Namespace("http://example.org/mae/thesaurus/")

def clean_label(label):
    return label.strip() if label else ""

def make_uri(prefix, id):
    return URIRef(f"{prefix}{id}")

## 2. Lecture et parsing du CSV (point-virgule, encodage Windows/accents)

In [6]:
def read_thesaurus_csv(csv_file):
    """
    Lecture du thésaurus au format CSV (point-virgule, encodage Windows),
    retourne une liste de concepts (dicts)
    """
    concepts = []
    for encoding in ('cp1252', 'utf-8-sig'):
        try:
            with open(csv_file, newline='', encoding=encoding) as f:
                reader = csv.DictReader(f, delimiter=';')
                for row in reader:
                    clean_row = {k: (v.strip() if isinstance(v, str) else "") for k, v in row.items()}
                    concepts.append(clean_row)
            break
        except UnicodeDecodeError:
            concepts = []
            continue
    if not concepts:
        raise UnicodeDecodeError("Impossible de lire le fichier avec cp1252 ni utf-8-sig.")
    print(f"{len(concepts)} concepts lus.")
    return concepts

concepts = read_thesaurus_csv(CSV_FILE)

27680 concepts lus.


## 3. Construction du graphe SKOS et de l'index d'arborescence

In [16]:
def build_skos_graph(concepts):
    g = Graph()
    g.bind("skos", SKOS)
    g.bind("ex", EX)
    g.bind("th", THESAURUS)
    id_to_uri = {}

    for c in concepts:
        # Générique
        gen_id = c["ID_TG"]
        gen_label = clean_label(c["TermeGen"])
        if gen_id and gen_label:
            gen_uri = make_uri(THESAURUS, gen_id)
            id_to_uri[gen_id] = gen_uri
            g.add((gen_uri, RDF.type, SKOS.Concept))
            g.add((gen_uri, SKOS.prefLabel, Literal(gen_label, lang="fr")))
        # Spécifique
        spe_id = c["ID_TS"]
        spe_label = clean_label(c["TermeSpe"])
        if spe_id and spe_label:
            spe_uri = make_uri(THESAURUS, spe_id)
            id_to_uri[spe_id] = spe_uri
            g.add((spe_uri, RDF.type, SKOS.Concept))
            g.add((spe_uri, SKOS.prefLabel, Literal(spe_label, lang="fr")))
            if gen_id and gen_label:
                g.add((spe_uri, SKOS.broader, gen_uri))
                g.add((gen_uri, SKOS.narrower, spe_uri))
            # Notes
            note = c.get("Note", "")
            if note:
                g.add((spe_uri, SKOS.note, Literal(note, lang="fr")))
            # EP
            ep = c.get("EP", "")
            if ep:
                for alt in ep.split("|"):
                    alt = alt.strip()
                    if alt:
                        g.add((spe_uri, SKOS.altLabel, Literal(alt, lang="fr")))
            # TA (termes associés)
            ta = c.get("TA", "")
            id_ta = c.get("ID_TA", "")
            if ta and id_ta:
                ta_labels = [x.strip() for x in ta.split("|")]
                ta_ids = [x.strip() for x in id_ta.split("|")]
                for tlabel, tid in zip(ta_labels, ta_ids):
                    if tid:
                        ta_uri = make_uri(THESAURUS, tid)
                        if (ta_uri, RDF.type, SKOS.Concept) not in g:
                            g.add((ta_uri, RDF.type, SKOS.Concept))
                            g.add((ta_uri, SKOS.prefLabel, Literal(tlabel, lang="fr")))
                        g.add((spe_uri, SKOS.related, ta_uri))
                        g.add((ta_uri, SKOS.related, spe_uri))
    print("Graphe SKOS construit.")
    return g, id_to_uri

skos_graph, id_to_uri = build_skos_graph(concepts)
# Export SKOS
skos_graph.serialize(destination=SKOS_FILE, format="turtle")
print(f"Exporté SKOS vers {SKOS_FILE}")

Graphe SKOS construit.
Exporté SKOS vers data/thesaurus.ttl


## 4. Visualisation : arbre et graphe complet

In [17]:
def build_networkx_graph(concepts):
    G = nx.DiGraph()
    id_to_label = {}
    for c in concepts:
        if c["ID_TG"]:
            id_to_label[c["ID_TG"]] = c["TermeGen"]
        if c["ID_TS"]:
            id_to_label[c["ID_TS"]] = c["TermeSpe"]
    for c in concepts:
        gen_id = c["ID_TG"]
        spe_id = c["ID_TS"]
        if gen_id and spe_id:
            G.add_edge(id_to_label[gen_id], id_to_label[spe_id], relation="broader")
        ta = c.get("TA", "")
        id_ta = c.get("ID_TA", "")
        if ta and id_ta and spe_id:
            ta_labels = [x.strip() for x in ta.split("|")]
            ta_ids = [x.strip() for x in id_ta.split("|")]
            for tlabel, tid in zip(ta_labels, ta_ids):
                if tid and tid in id_to_label:
                    G.add_edge(id_to_label[spe_id], id_to_label[tid], relation="related")
    print(f"Graphe de visualisation construit : {G.number_of_nodes()} noeuds, {G.number_of_edges()} arêtes.")
    return G

G = build_networkx_graph(concepts)
# Arbre (relations broader seulement)
def plot_tree(G):
    T = nx.DiGraph()
    for u, v, d in G.edges(data=True):
        if d["relation"] == "broader":
            T.add_edge(u, v)
    plt.figure(figsize=(13, 6))
    pos = nx.shell_layout(T)
    nx.draw(T, pos, with_labels=True, arrows=True, node_color='lightblue', node_size=1000, font_size=9)
    plt.title("Arbre hiérarchique (générique > spécifique)")
    plt.show()
plot_tree(G)

Graphe de visualisation construit : 21519 noeuds, 26554 arêtes.


In [9]:
# Graphe complet
def plot_graph(G):
    plt.figure(figsize=(15, 8))
    pos = nx.spring_layout(G, seed=42, k=0.25)
    edge_colors = ["blue" if d["relation"]=="broader" else "orange" for u,v,d in G.edges(data=True)]
    nx.draw(G, pos, with_labels=True, arrows=True, node_color='lightgreen', edge_color=edge_colors, node_size=1000, font_size=9)
    plt.title("Thésaurus : génériques et associés")
    plt.show()
plot_graph(G)

KeyboardInterrupt: 

<Figure size 1500x800 with 0 Axes>

## 5. Export CSV arborescence (niveau 1, niveau 2, ...)
Chaque ligne = un concept spécifique (ID_TS), chaque colonne = un niveau d'arborescence.

In [ ]:
# Dictionnaires pour reconstituer l'arborescence
id_to_label = {}
child_to_parent = {}
# Index rapide des relations
for c in concepts:
    if c["ID_TG"]:
        id_to_label[c["ID_TG"]] = c["TermeGen"]
    if c["ID_TS"]:
        id_to_label[c["ID_TS"]] = c["TermeSpe"]
    if c["ID_TS"] and c["ID_TG"]:
        child_to_parent[c["ID_TS"]] = c["ID_TG"]

def get_niveaux(concept_id):
    chemin = []
    while concept_id in id_to_label:
        chemin.insert(0, id_to_label[concept_id])
        if concept_id in child_to_parent:
            concept_id = child_to_parent[concept_id]
        else:
            break
    return chemin

# Liste des chemins (une ligne par ID_TS)
chemins = []
for c in concepts:
    if c["ID_TS"]:
        niveaux = get_niveaux(c["ID_TS"])
        chemins.append(niveaux)

max_niveaux = max(len(chemin) for chemin in chemins)

# Création d'un DataFrame pour export
df = pd.DataFrame([([""] * (max_niveaux - len(l)) + l) for l in chemins], columns=[f"niveau {i+1}" for i in range(max_niveaux)])
df.to_csv(ARBO_CSV, index=False, encoding="utf-8")
print(f"Export arborescence terminé : {ARBO_CSV}")
df.head()